![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [2]:
// We need to load assemblies at the start in their own cell
#load "../Initialize.csx"
#load "../QuantConnect.csx"

using System;
using System.Linq;
using System.Collections.Generic;

using QuantConnect;
using QuantConnect.Data.Market;
using QuantConnect.Research;

In [3]:
using System;
using System.Linq;
using System.Collections.Generic;
using Newtonsoft.Json;

// =====================================
// Helpers
// =====================================

decimal Round2(decimal value) => Math.Round(value, 2);

decimal Median(List<decimal> values)
{
    if (values == null || values.Count == 0) return 0m;
    var sorted = values.OrderBy(x => x).ToList();
    int n = sorted.Count;
    int mid = n / 2;
    return (n % 2 == 0)
        ? (sorted[mid - 1] + sorted[mid]) / 2m
        : sorted[mid];
}

decimal StdDev(List<decimal> values)
{
    if (values == null || values.Count == 0) return 0m;
    var mean = values.Average();
    var sumSq = values.Sum(v => (v - mean) * (v - mean));
    return (decimal)Math.Sqrt((double)(sumSq / values.Count));
}

// =====================================
// DTO for JSON output (camelCase)
// =====================================

// =====================================
// DailyBarRecord definition
// =====================================

public class DailyBarRecord
{
    public string  date        { get; set; }
    public decimal open        { get; set; }
    public decimal high        { get; set; }
    public decimal low         { get; set; }
    public decimal close       { get; set; }
    public decimal volume      { get; set; }
    public decimal openToHigh  { get; set; }
    public decimal openToLow   { get; set; }
    public decimal openToClose { get; set; }
    public decimal totalDelta  { get; set; }
}


In [11]:
// =====================================
// QuantBook setup
// =====================================

var qb = new QuantBook();

const string TickerSymbol = "QQQ";
const int TotalDeltaOutlierThreshold = 10;

var HistoryStartDate = new DateTime(1999, 3, 10);
var HistoryEndDate   = new DateTime(2025, 12, 1);

// Add equity
var equity = qb.AddEquity(TickerSymbol, Resolution.Daily);

// Use SplitAdjusted normalization (will still be cleaned by outlier filters)
equity.SetDataNormalizationMode(DataNormalizationMode.SplitAdjusted);

// Load daily bars
var allDailyBars = qb.History<TradeBar>(
    equity.Symbol,
    HistoryStartDate,
    HistoryEndDate,
    Resolution.Daily
)
.OrderBy(b => b.EndTime)
.ToList();

// =====================================
// Build DAILY records + filter on totalDelta
// =====================================

var dailyRecords = new List<DailyBarRecord>();
var filteredDailyOutliers = new List<TradeBar>();

foreach (var bar in allDailyBars.OrderBy(b => b.Time))
{
    decimal open  = bar.Open;
    decimal high  = bar.High;
    decimal low   = bar.Low;
    decimal close = bar.Close;

    // Basic sanity: non-positive prices are junk
    if (open <= 0 || high <= 0 || low <= 0 || close <= 0)
    {
        filteredDailyOutliers.Add(bar);
        continue;
    }

    // Percentages
    decimal pctOpenToHigh  = open != 0 ? (high - open)  / open * 100m : 0m;
    decimal pctOpenToLow   = open != 0 ? (low  - open)  / open * 100m : 0m;
    decimal pctOpenToClose = open != 0 ? (close - open) / open * 100m : 0m;

    // totalDelta:
    //  - If the bar is green (close > open): % change low -> high
    //  - If the bar is red   (close < open): % change high -> low (negative)
    //  - If open == close: 0
    decimal pctTotalDelta = 0m;

    if (close > open)
    {
        // Green bar: (high - low) / low * 100
        pctTotalDelta = (high - low) / low * 100m;
    }
    else if (close < open)
    {
        // Red bar: (low - high) / high * 100 (negative)
        pctTotalDelta = (low - high) / high * 100m;
    }

    // Filter criterion based on totalDelta: remove bars where |totalDelta| > 15%
    if (Math.Abs(pctTotalDelta) > TotalDeltaOutlierThreshold)
    {
        filteredDailyOutliers.Add(bar);
        continue;
    }

    // Keep this bar
    dailyRecords.Add(new DailyBarRecord
    {
        date        = bar.Time.Date.ToString("yyyy-MM-dd"),
        open        = Round2(open),
        high        = Round2(high),
        low         = Round2(low),
        close       = Round2(close),
        volume      = Round2(bar.Volume),
        openToHigh  = Round2(pctOpenToHigh),
        openToLow   = Round2(pctOpenToLow),
        openToClose = Round2(pctOpenToClose),
        totalDelta  = Round2(pctTotalDelta)
    });
}

// =====================================
// Compute statistics (DAILY)
// =====================================

var openToHighValues  = dailyRecords.Select(r => r.openToHigh).ToList();
var openToLowValues   = dailyRecords.Select(r => r.openToLow).ToList();
var openToCloseValues = dailyRecords.Select(r => r.openToClose).ToList();
var totalDeltaValues  = dailyRecords.Select(r => r.totalDelta).ToList();

Console.WriteLine("Daily Percentage Move Statistics (2-decimal rounded):\n");

// openToHigh
Console.WriteLine(
    $"openToHigh  -> mean: {Round2(openToHighValues.Average())}%, " +
    $"median: {Round2(Median(openToHighValues))}%, " +
    $"std dev: {Round2(StdDev(openToHighValues))}%");

// openToLow
Console.WriteLine(
    $"openToLow   -> mean: {Round2(openToLowValues.Average())}%, " +
    $"median: {Round2(Median(openToLowValues))}%, " +
    $"std dev: {Round2(StdDev(openToLowValues))}%");

// openToClose
Console.WriteLine(
    $"openToClose -> mean: {Round2(openToCloseValues.Average())}%, " +
    $"median: {Round2(Median(openToCloseValues))}%, " +
    $"std dev: {Round2(StdDev(openToCloseValues))}%");

// totalDelta
Console.WriteLine(
    $"totalDelta  -> mean: {Round2(totalDeltaValues.Average())}%, " +
    $"median: {Round2(Median(totalDeltaValues))}%, " +
    $"std dev: {Round2(StdDev(totalDeltaValues))}%");

Console.WriteLine();

// =====================================
// Report filtered DAILY outliers (with signed totalDelta)
// =====================================

if (filteredDailyOutliers.Count > 0)
{
    Console.WriteLine("Filtered DAILY outlier bars (with signed totalDelta):");
    foreach (var b in filteredDailyOutliers.OrderBy(x => x.Time))
    {
        decimal open  = b.Open;
        decimal high  = b.High;
        decimal low   = b.Low;
        decimal close = b.Close;

        decimal pctTotalDelta = 0m;
        bool hasTotalDelta = true;

        // If prices are invalid, we can't compute totalDelta meaningfully
        if (open <= 0 || high <= 0 || low <= 0 || close <= 0)
        {
            hasTotalDelta = false;
        }
        else if (close > open)
        {
            // Green bar: (high - low) / low * 100
            pctTotalDelta = (high - low) / low * 100m;
        }
        else if (close < open)
        {
            // Red bar: (low - high) / high * 100 (negative)
            pctTotalDelta = (low - high) / high * 100m;
        }
        else
        {
            // Flat bar
            pctTotalDelta = 0m;
        }

        string tdText = hasTotalDelta
            ? $"{pctTotalDelta:F2}%"
            : "N/A";

        Console.WriteLine(
            $"{b.Time:yyyy-MM-dd} | " +
            $"O={b.Open} H={b.High} L={b.Low} C={b.Close} V={b.Volume} " +
            $"totalDelta={tdText}"
        );
    }
}
else
{
    Console.WriteLine("No DAILY outlier bars were filtered.");
}

Console.WriteLine();

// =====================================
// Build JSON structures for filtered data
// =====================================

// Daily filtered bars -> full bar info
var filteredDayRecords = filteredDailyOutliers
    .OrderBy(b => b.Time)
    .Select(b => new
    {
        date   = b.Time.Date.ToString("yyyy-MM-dd"),
        open   = Round2(b.Open),
        high   = Round2(b.High),
        low    = Round2(b.Low),
        close  = Round2(b.Close),
        volume = Round2(b.Volume)
    })
    .ToList();

// =====================================
// Final JSON object (top-level)
// =====================================

var finalObject = new
{
    ticker = TickerSymbol,
    analysisStartDate = HistoryStartDate.ToString("yyyy-MM-dd"),
    analysisEndDate   = HistoryEndDate.ToString("yyyy-MM-dd"),
    numDays           = dailyRecords.Count,
    numFilteredDays   = filteredDayRecords.Count,
    
    // Filtered daily outliers BEFORE main daily data
    totalDeltaThreshold = TotalDeltaOutlierThreshold,
    filteredDays = filteredDayRecords,

    dailyData = dailyRecords
};

// Output JSON
JsonConvert.SerializeObject(finalObject, Formatting.Indented)